# 다중 선형 회귀 Multiple Linear Regression


여러개의 독립변수를 가지는 예측모델을 만들 때, 사용
이번에는 공부시간, 결석횟수, 공부장소 세가지 독립변수를 이용하여 종속변수y를 예측하는 모델을 구상한다.


## 원-핫 인코딩

공부장소처럼, 숫자로 표현하기 힘든 변수의 경우 
더미 컬럼을 생성하여 1과 0으로만 표현하는 방법 
1 = 내가 표현하고 싶은 값
0 = 나머지


## 다중공선성

독립 변수들 간에 서로 강한 상관관계를 가지면서 회귀계수 추정의 오류가 나타나는 문제


이를 해결하기 위해서, dummy column 을 n-1개만 사용한다.

## 원-핫 인코딩 실습

In [1]:
import pandas as pd

In [4]:
dataset = pd.read_csv('./data/ScikitLearn/MultipleLinearRegressionData.csv')
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

In [7]:
X

array([[0.5, 3, 'Home'],
       [1.2, 4, 'Library'],
       [1.8, 2, 'Cafe'],
       [2.4, 0, 'Cafe'],
       [2.6, 2, 'Home'],
       [3.2, 0, 'Home'],
       [3.9, 0, 'Library'],
       [4.4, 0, 'Library'],
       [4.5, 5, 'Home'],
       [5.0, 1, 'Cafe'],
       [5.3, 2, 'Cafe'],
       [5.8, 0, 'Cafe'],
       [6.0, 3, 'Library'],
       [6.1, 1, 'Cafe'],
       [6.2, 1, 'Library'],
       [6.9, 4, 'Home'],
       [7.2, 2, 'Cafe'],
       [8.4, 1, 'Home'],
       [8.6, 1, 'Library'],
       [10.0, 0, 'Library']], dtype=object)

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
### transformers = 전달값
### 첫번째 인자 = 어떤 방식 ? encoder
### 두번째 인자 = 인코더를 수행할 클래스 객체
### OneHotEncoder
### drop인자 = 빼고 처리할것,first 를 넣어줘서 첫번째건 빼고 처리함. 
##### beacause 다중공산성 문제 해결을 위함
### 2번째 index(장소)에 대해서 OneHotEncoder 수행
### remainder = 'passthrough' 적용하지 않는 나머지 데이터는 그대로 둔다.

ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(drop='first'),[2])],remainder='passthrough')
X = ct.fit_transform(X)
X

array([[1.0, 0.0, 0.5, 3],
       [0.0, 1.0, 1.2, 4],
       [0.0, 0.0, 1.8, 2],
       [0.0, 0.0, 2.4, 0],
       [1.0, 0.0, 2.6, 2],
       [1.0, 0.0, 3.2, 0],
       [0.0, 1.0, 3.9, 0],
       [0.0, 1.0, 4.4, 0],
       [1.0, 0.0, 4.5, 5],
       [0.0, 0.0, 5.0, 1],
       [0.0, 0.0, 5.3, 2],
       [0.0, 0.0, 5.8, 0],
       [0.0, 1.0, 6.0, 3],
       [0.0, 0.0, 6.1, 1],
       [0.0, 1.0, 6.2, 1],
       [1.0, 0.0, 6.9, 4],
       [0.0, 0.0, 7.2, 2],
       [1.0, 0.0, 8.4, 1],
       [0.0, 1.0, 8.6, 1],
       [0.0, 1.0, 10.0, 0]], dtype=object)

위를 보면 앞쪽에 두 개의 컬럼이 생성됨을 볼 수 있다.
1 , 0 = Home
0 , 1 = Library
0 , 0 = cafe 가 된 것을 확인 가능하다.


위와 같은 처리를 통해 범주형 데이터의 처리가 가능하다

### 데이터 세트 분리

In [16]:
# 데이터 세트 분리를 위환 모듈
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)


## 학습 (다중 선형 회귀)

In [19]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

### 예측 값과 실제 값 비교 (테스트 세트)

In [20]:
y_pred = reg.predict(X_test)
y_pred


array([ 92.15457859,  10.23753043, 108.36245302,  38.14675204])

In [21]:
y_test

array([ 90,   8, 100,  38])

In [22]:
reg.coef_

array([-5.82712824, -1.04450647, 10.40419528, -1.64200104])

위 결과값을 보면
[집,도서관,공부시간,결석회수]임을 알 수 있다.<br>
카페는 따로 안나오지만 0을 기본값으로 가진다고 보면 된다.<br>
집 < 도서관 < 카페 순으로 공부 효율이 높고<br>
공부시간이 1시간 증가할 때마다 점수는 약 10점씩 오르고,<br>
결석 1회당 약 -1.6점씩 점수에 영향을 준다고 해석 가능하다.<br>

In [23]:
reg.intercept_

5.365006706544733

### 모델 평가


#### 기준,
testset을 평가한 점수보다 trainset을 평가한 점수가 살짝 작게 나오면 좋은 모델이다.

In [24]:
reg.score(X_train,y_train)

0.9623352565265528

In [25]:
reg.score(X_test,y_test)

0.9859956178877445

### 다양한 평가 지표(회귀 모델)

MAE(Mean Absolute Error) : |실제값 - 예측값|<br>
MSE(Mean Squared Error) : (실제값 - 예측값)^2<br>
R2 : 결정 계수<br>
<br>
R2 는 1에 가까울수록 좋고 나머지는 0에 가까울수록 좋다<br>

In [27]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test,y_pred) # 실제값과 예측값의 mae


3.225328518828811

In [30]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,y_pred) #MSE

19.90022698151514

In [31]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,y_pred,squared=False) #RMSE

4.460967045553591

In [32]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred) #R2

## R2는 기본 score()함수에 사용되는 평가지표임

0.9859956178877445